In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import get_vegetation_periods, create_binary_ndvi_indicator, create_binary_mask

In [3]:
ee.Initialize(project="thurgau-irrigation")

In [4]:
yr = "2022"
time_intervals = extract_time_ranges(
    time_range=[yr + "-01-01", yr + "-12-31"], agg_interval=15
)
print(time_intervals.getInfo())

# 1519862400000
# 1646092800000

[[{'type': 'Date', 'value': 1640995200000}, {'type': 'Date', 'value': 1642334400000}], [{'type': 'Date', 'value': 1642334400000}, {'type': 'Date', 'value': 1643673600000}], [{'type': 'Date', 'value': 1643673600000}, {'type': 'Date', 'value': 1644883200000}], [{'type': 'Date', 'value': 1644883200000}, {'type': 'Date', 'value': 1646092800000}], [{'type': 'Date', 'value': 1646092800000}, {'type': 'Date', 'value': 1647432000000}], [{'type': 'Date', 'value': 1647432000000}, {'type': 'Date', 'value': 1648771200000}], [{'type': 'Date', 'value': 1648771200000}, {'type': 'Date', 'value': 1650067200000}], [{'type': 'Date', 'value': 1650067200000}, {'type': 'Date', 'value': 1651363200000}], [{'type': 'Date', 'value': 1651363200000}, {'type': 'Date', 'value': 1652702400000}], [{'type': 'Date', 'value': 1652702400000}, {'type': 'Date', 'value': 1654041600000}], [{'type': 'Date', 'value': 1654041600000}, {'type': 'Date', 'value': 1655337600000}], [{'type': 'Date', 'value': 1655337600000}, {'type': '

In [5]:
test_aoi2 = ee.Geometry.Polygon(
    [
        [
            [8.737412756413452, 47.648802254332814],
            [8.737412756413452, 47.61548626028342],
            [8.829423254460327, 47.61548626028342],
            [8.829423254460327, 47.648802254332814],
        ]
    ]
)

In [6]:
# filtered_sentinel_data = get_harmonic_ts(year=int(yr), aoi=test_aoi2, time_intervals=time_intervals)

# # Print the timestamp and the NDVI value of the first 5 elements in the collection
# print(filtered_sentinel_data.first().getInfo())

In [7]:
# filtered_image = filtered_sentinel_data.first().select("t")

# # Print a few values to check if the data is correct
# print(filtered_image.reduceRegion(reducer=ee.Reducer.first(), geometry=test_aoi2, scale=10).getInfo())

In [8]:
result = get_vegetation_periods(2022, test_aoi2, time_intervals)

# the the band names of the image
print(result)

Year 2022: Data size after interpolation: 24
{'first': <ee.image.Image object at 0x16d9c4250>, 'last': <ee.image.Image object at 0x16d9c7450>, 'double': <ee.image.Image object at 0x16db77050>, 'first2': <ee.image.Image object at 0x16d9c5010>, 'last2': <ee.image.Image object at 0x16d9c6f10>}


In [9]:
double_cropping = ee.Image(result.get("double"))

Map = geemap.Map()
Map.centerObject(test_aoi2, 13)
params = {'min': 0, 'max': 1, 'palette': ['white', 'black']}
Map.addLayer(double_cropping, params, 'Double Cropping', True)

Map

Map(center=[47.632151683310106, 8.783418005435093], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.toList(99).get(4))
Map.centerObject(test_aoi2, 13)
ndvi_params = {'bands': ['NDVI'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

NameError: name 'filtered_sentinel_data' is not defined